In [1]:
import pandas as pd
import numpy as np
anime_df = pd.read_csv('anime.csv')
rating_df = pd.read_csv('rating.csv')


In [2]:
n_ratings = len(rating_df)
n_anime = len(rating_df['anime_id'].unique())
n_users = len(rating_df['user_id'].unique())
print(f"The No. Of Anime = {n_anime}\nNo. of users = {n_users}")

The No. Of Anime = 11200
No. of users = 73515


In [ ]:
from scipy.sparse import csr_matrix
N = len(rating_df['user_id'].unique()) # 73515 No. Of users in data
M = len(rating_df['anime_id'].unique()) # 11200 No. of anime in data

user_mapper = dict(zip(np.unique(rating_df["user_id"]), list(range(N)))) #Mapping each used_id to a number in a sorted order in a dictionary
anime_mapper = dict(zip(np.unique(rating_df["anime_id"]), list(range(M))))#Mapping each anime_id to a number in a sorted order in a dictionary 

#user_inv_mapper = dict(zip(list(range(N)), np.unique(rating_df["user_id"])))#Mapping each number to a user_id in a sorted order in a dictionary
anime_inv_mapper = dict(zip(list(range(M)), np.unique(rating_df["anime_id"])))#Mapping each number to a anime_id in a sorted order in a dictionary

user_index = [user_mapper[i] for i in rating_df['user_id']] #Row for sparse matridx
anime_index = [anime_mapper[i] for i in rating_df['anime_id']] #Column for sparse matrix

X = csr_matrix((rating_df["rating"], (anime_index, user_index)), shape=(M, N))#Where anime_index = row, user_index = column and rating_df = the data

nova = pd.DataFrame(X.todense()) #converts the sparse matrix to a data frame for easier visualization

nova.head()

In [104]:
from sklearn.neighbors import NearestNeighbors
anime_id = 5114 #5114 #Anime_id for Full Metal Alchemist
k=11 # Top n suggesting anime
neighbour_ids = [] # List to store recommended anime ids
anime_ind = anime_mapper[anime_id]  #Storing the Value of the anime_id key in dict which can be used in anime_inv_mapper
anime_vec = X[anime_ind] # Gets the ratings for the user watched anime given by 73k users
kNN = NearestNeighbors(n_neighbors=k, algorithm="auto", metric='cosine') #Making K Nearest neighbors object
kNN.fit(X) #Fitting the ratings of each anime in the KNN object
neighbour = kNN.kneighbors(anime_vec, return_distance=False) # Stores an array of K nearest neighbors.
print(neighbour)

for i in range(0,11):
    n = neighbour.item(i)
    neighbour_ids.append(anime_inv_mapper[n])
#Stores the ids of the neighbors according to the data. as neighbors contains the number not id
neighbour_ids.pop(0)

[[3965   99 1389 7439 1427 2651 5673 6606 4630 1816 5968]]


5114

In [106]:
anime_titles = dict(zip(anime_df['anime_id'], anime_df['name'])) #Creates a dict containing the ids and anime names

print(f"Since you watched {anime_titles[anime_id]} we would suggest watching\n")
for i, j in enumerate(neighbour_ids, start=1):
    print(f"{i}){anime_titles[j]}")

Since you watched Fullmetal Alchemist: Brotherhood we would suggest watching

1)Fullmetal Alchemist
2)Death Note
3)Shingeki no Kyojin
4)Code Geass: Hangyaku no Lelouch
5)Code Geass: Hangyaku no Lelouch R2
6)Steins;Gate
7)Sword Art Online
8)Angel Beats!
9)Tengen Toppa Gurren Lagann
10)Ao no Exorcist
